<a href="https://colab.research.google.com/github/SumairaAmbreen/Contrroller/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Step 1: Convert categorical data to numbers
dataset['sex'] = dataset['sex'].map({'male': 0, 'female': 1})
dataset['smoker'] = dataset['smoker'].map({'yes': 1, 'no': 0})
dataset['region'] = dataset['region'].map({'southwest': 0, 'southeast': 1, 'northwest': 2, 'northeast': 3})


In [ ]:
# Step 2: Split data into train and test datasets
train_dataset = dataset.sample(frac=0.8, random_state=123)  # 80% for training
test_dataset = dataset.drop(train_dataset.index)  # Remaining 20% for testing

In [ ]:
# Step 3: Separate labels (expenses) from features
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')


In [ ]:
# Step 4: Normalize the data
train_stats = train_dataset.describe().transpose()
def normalize(x):
    return (x - train_stats['mean']) / train_stats['std']

train_dataset = normalize(train_dataset)
test_dataset = normalize(test_dataset)



In [ ]:
# Step 5: Build the model
def build_model():
    model = keras.Sequential([
    layers.Input(shape=(train_dataset.shape[1],)),
    layers.Dense(128, activation='relu'),
    # layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
    ])


    optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)

    model.compile(optimizer=optimizer,
                  loss='mse',
                  metrics=['mae', 'mse'])
    return model



model = build_model()

In [ ]:
# Step 6: Train the model
history = model.fit(
    train_dataset,
    train_labels,
    epochs=100,
    batch_size=64,  # Try values like 16, 32, or 64
    validation_split=0.2,
    verbose=0,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)]
)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
